In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
!pip3 install torch
!pip3 install transformers
!pip3 install emoji==0.6.0

In [ ]:
pip install openai==0.28

In [ ]:
!pip install transformers
!pip install torch
!pip install google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
!pip install --upgrade tweety-ns

In [2]:
from tweety import Twitter
from pprint import pprint

username = "" # Enter your username
password = "" # Enter your twitter password
app = Twitter("session")
app.sign_in(username,password)

User(id=1768208341547859968, username=SaarTestmail, name=Test Gmail, verified=False)

In [3]:
target_username = "cars24india"
num_pages = 1

user = app.get_user_info(target_username)
all_tweets = app.get_tweets(user, pages=num_pages, replies=True)
all_threads = []
x = 0
for tweet in all_tweets:
    x+=1
    try: 
        thread = []
        for reply in tweet.tweets:
            if reply.author.username == target_username:
                continue
            rep = {}
            rep["created_at"] = reply.created_on 
            rep["user_name"] = reply.author.username
            rep["text"] = reply.text
            rep["email"] = reply.author.screen_name + "@example.com"
            thread.append(rep)
        print("Thread", x)
        all_threads+=thread
    except:
        continue
pprint(all_threads)

Thread 1
Thread 2
Thread 3
Thread 4
Thread 5
Thread 6
Thread 7
Thread 8
Thread 9
Thread 10
Thread 11
Thread 12
Thread 13
Thread 14
Thread 15
Thread 16
Thread 17
Thread 18
Thread 19
[{'created_at': datetime.datetime(2024, 3, 16, 10, 25, 6, tzinfo=tzutc()),
  'email': 'kedarjoshi@example.com',
  'text': 'Now @cars24india is using prerecorded voice messages for their '
          'latest endeavor in DND violations.\n'
          '\n'
          '@TRAI',
  'user_name': 'kedarjoshi'},
 {'created_at': datetime.datetime(2024, 3, 15, 11, 23, 45, tzinfo=tzutc()),
  'email': 'ArbindSikarwar@example.com',
  'text': '@cars24india \n'
          'Absolutely pathetic valuation.\n'
          'Looks like you guys have no idea about car pricing.\n'
          '#stoplooting \n'
          "You should shut the business if you can't value people's time and "
          'money. Quoting ridiculous price is just not acceptable.\n'
          '#boycottcars24',
  'user_name': 'ArbindSikarwar'},
 {'created_at': datetim

In [4]:
import os
import base64
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from transformers import pipeline

SCOPES = ['https://www.googleapis.com/auth/gmail.send']
CLIENT_SECRET_FILE = 'credentials.json' 
TOKEN_FILE = 'token.json'

def get_credentials():
    
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
    return creds

def create_message(sender, to, subject, message_text):
   
    message = MIMEMultipart()
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    msg = MIMEText(message_text)
    message.attach(msg)
    raw_message = base64.urlsafe_b64encode(message.as_bytes())
    raw_message = raw_message.decode('utf-8')
    return {'raw': raw_message}

def send_message(service, user_id, message):
   
    try:
        message = service.users().messages().send(userId=user_id, body=message).execute()
        print('Message Id: %s' % message['id'])
        return message
    except Exception as e:
        print('An error occurred: %s' % e)
        return None

def analyze_sentiment(text):
   
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    result = sentiment_analyzer(text)
    sentiment_label = result[0]['label']
    print(text,sentiment_label)
    return sentiment_label == 'NEGATIVE'

# def analyze_sentiment(text):
#     # Function to analyze sentiment using a pre-trained model
#     pipe = pipeline("text-classification", model="Hate-speech-CNERG/dehatebert-mono-english")
#     result = pipe(text)
#     sentiment_la 'HATE'bel = result[0]['label']
#     print(text,sentiment_label)
#     return sentiment_label == "HATE"

def main():
    sender = "" # Enter your mail id

    post_owner_email = "post_owner_mail@example.com"

    creds = get_credentials()
    service = build('gmail', 'v1', credentials=creds)

    
    comments_data = all_threads
     
    for comment_data in comments_data:
        comment = comment_data['text']
        sentiment = analyze_sentiment(comment)
        if sentiment:
         
            user_email = comment_data['email']
            user_subject = "Warning: Negative Sentiment Detected"
            user_message = f"Hello {comment_data['user_name']},\n\nWe have detected negative sentiment associated with your comment: '{comment}'. Please review the content.\n\nThank you."
            user_alert = create_message(sender, user_email, user_subject, user_message)
            send_message(service, 'me', user_alert)
            
        
            post_owner_subject = "Notification: Negative Comment Alert"
            post_owner_message = f"Hello,\n\nThe user {comment_data['user_name']} has posted a negative comment: '{comment}'. Please take necessary actions as required.\n\nThank you."
            post_owner_alert = create_message(sender, post_owner_email, post_owner_subject, post_owner_message)
            send_message(service, 'me', post_owner_alert)
        else:
            print("Mail not send")

if __name__ == '__main__':
    main()

Now @cars24india is using prerecorded voice messages for their latest endeavor in DND violations.

@TRAI NEGATIVE
Message Id: 18e4a43934246c6c
Message Id: 18e4a4394a432708
@cars24india 
Absolutely pathetic valuation.
Looks like you guys have no idea about car pricing.
#stoplooting 
You should shut the business if you can't value people's time and money. Quoting ridiculous price is just not acceptable.
#boycottcars24 NEGATIVE
Message Id: 18e4a43a5406a3a2
Message Id: 18e4a43a92f3fabe
@cars24india sell me #fraud car with fraud documents Fake NOC @AgrawalMehul @gajen_jangid @vikramchopra recover agent insulting my mother and snatched my car even i was showing all documents #Justice #scams #HighCourt #ConsumerRights #consumercourt #noidapolice #Liar be aware NEGATIVE
Message Id: 18e4a43c5f6e5e3c
Message Id: 18e4a43c9520ae8f
@cars24india what's the update on the RC changing .. it's almost 3 months and no response .. NEGATIVE
Message Id: 18e4a43df53c51fa
Message Id: 18e4a43e2bb1a940
After a y